<a href="https://colab.research.google.com/github/shrutisj12/AgriSense-AI-Intelligent-Crop-Disease-Assistant/blob/main/Phase_3_CHATBOT_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AGRISENSE AI CHATBOT

In [1]:
%%writefile app.py
"""
=============================================================================
                        🌾 AGRISENSE AI CHATBOT (Phase 3)
=============================================================================
An intelligent agricultural assistant for Indian farmers
Supports: Hindi, Hinglish, English
Region-aware | Crop-specific | Expert + AI hybrid responses

Author: AgrisenseAI Team
Version: 3.0 (Streamlit Edition)
=============================================================================
"""

# =============================================================================
# 1. INSTALL / IMPORT LIBRARIES
# =============================================================================
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, warnings

warnings.filterwarnings("ignore")

# =============================================================================
# 2. PAGE CONFIGURATION
# =============================================================================
st.set_page_config(
    page_title="AgriSense AI 🌾",
    page_icon="🌱",
    layout="centered"
)

st.title("🌾 **AgriSense AI – Indian Agriculture Assistant**")
st.markdown("### 👨‍🌾 Bilingual (English / Hindi / Hinglish) Smart Chatbot")

# =============================================================================
# 3. LOAD MODEL (Once)
# =============================================================================
@st.cache_resource(show_spinner=False)
def load_model():
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float32)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    return tokenizer, model, device

with st.spinner("⏳ Loading AgriSense AI model..."):
    tokenizer, model, device = load_model()
st.success("✅ Model loaded successfully!")

# =============================================================================
# 4. STATE & EXPERT DATABASE (shortened example for brevity)
# =============================================================================
STATE_CROP_DATABASE = {
    "punjab": {
        "crops": ["wheat", "rice", "maize", "cotton"],
        "soil": "Alluvial (fertile, loamy)",
        "varieties": {
            "wheat": "HD-3086, PBW-725",
            "rice": "PR-126, Pusa-44",
            "maize": "PMH-1, DHM-117",
            "cotton": "RCH-2 BG II, MRC-7361"
        },
        "tips": "Use Happy Seeder for stubble management and conserve water."
    },
    "maharashtra": {
        "crops": ["cotton", "sugarcane", "soybean"],
        "soil": "Black cotton soil (Vertisol)",
        "varieties": {
            "cotton": "NHH-44, Bunny BG II",
            "sugarcane": "Co-86032, CoM-0265",
            "soybean": "JS-335, MAUS-71"
        },
        "tips": "Avoid waterlogging; drip irrigation saves 40% water."
    }
}

EXPERT_ANSWERS = {
    "yellow_leaves": {
        "keywords": ["yellow", "yellowing", "पीली", "leaves turning yellow"],
        "answer": """**Yellow Leaves – Causes & Fix:**
1️⃣ **Nitrogen deficiency** – Spray 1% urea solution
2️⃣ **Overwatering** – Improve drainage
3️⃣ **Iron deficiency** – Spray Ferrous Sulphate 0.5%
4️⃣ **Disease** – Spray Mancozeb @2g/L
💡 Keep soil pH 6.0–6.8 and avoid waterlogging."""
    },
    "crop_rotation": {
        "keywords": ["crop rotation", "fasal chakra", "rotation"],
        "answer": """**Crop Rotation Benefits:**
🌱 Improves soil fertility
🐛 Reduces pests/diseases
🌾 Boosts yield 15–20%
**Pattern:** Wheat → Chickpea → Mustard"""
    }
}

# =============================================================================
# 5. CORE LOGIC FUNCTIONS
# =============================================================================
def detect_state_and_crop(query):
    q = query.lower()
    state, crop = None, None
    for s in STATE_CROP_DATABASE.keys():
        if s in q:
            state = s
            for c in STATE_CROP_DATABASE[s]["crops"]:
                if c in q:
                    crop = c
            break
    return state, crop


def check_expert_answer(query):
    q = query.lower()
    for topic, data in EXPERT_ANSWERS.items():
        matches = sum(1 for kw in data["keywords"] if kw in q)
        if matches >= max(1, len(data["keywords"]) * 0.5):
            return data["answer"]
    return None


def generate_with_ai(prompt, max_tokens=200):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            num_beams=2,
            no_repeat_ngram_size=3,
            repetition_penalty=2.0,
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        return f"⚠️ Error generating AI response: {e}"


def get_agrisense_response(query, language="hinglish"):
    expert = check_expert_answer(query)
    if expert:
        return f"📚 **Expert Answer:**\n\n{expert}"

    state, crop = detect_state_and_crop(query)
    q = query.lower()
    if state and any(k in q for k in ["best crop", "which crop", "recommend"]):
        d = STATE_CROP_DATABASE[state]
        res = f"📚 **Best Crops for {state.title()}**\n\n"
        res += f"**Soil Type:** {d['soil']}\n"
        res += f"**Top Crops:** {', '.join(d['crops'])}\n"
        res += f"**Tips:** {d['tips']}"
        return res

    if language == "hindi":
        prompt = f"Hindi mein 5-6 sentences mein practical krishi salah do:\n{query}"
    elif language == "hinglish":
        prompt = f"Hinglish (Hindi+English) mein simple 5-6 sentences mein salah do:\n{query}"
    else:
        prompt = f"Give simple English advice (5-6 sentences):\n{query}"

    ai_response = generate_with_ai(prompt)
    return f"📚 **Expert Answer:**\n\n{ai_response}"


# =============================================================================
# 6. UI ELEMENTS
# =============================================================================
with st.sidebar:
    st.header("⚙️ Settings")
    lang = st.radio("Select Language", ["English", "Hindi", "Hinglish"], index=2)
    st.info("💡 Type crop, state, or issue-based questions!\nExample: *My tomato leaves are yellow.*")

# Session state for chat
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Chat display
st.divider()
user_input = st.chat_input("Ask your question... (e.g., My wheat leaves are yellow)")

if user_input:
    with st.spinner("🤔 Thinking..."):
        response = get_agrisense_response(user_input, lang.lower())
    st.session_state.chat_history.append(("👨‍🌾 You", user_input))
    st.session_state.chat_history.append(("🤖 AgriSense AI", response))

# Display chat history
for role, msg in st.session_state.chat_history:
    if role.startswith("👨‍🌾"):
        st.markdown(f"**{role}:** {msg}")
    else:
        st.markdown(f"{role}: {msg}")

# =============================================================================
# 7. FOOTER
# =============================================================================
st.divider()
st.caption("🌱 *AgriSense AI v3.0 — Empowering Indian Agriculture with AI*")


Writing app.py


In [2]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.1 MB/s eta 0:00:00


In [3]:
from pyngrok import ngrok

# 🔑 Replace the string below with YOUR token
ngrok.set_auth_token("YOUR Token")

In [4]:
import threading, time
from pyngrok import ngrok
import random # Import random to generate a unique name

ngrok.kill()
time.sleep(5)  # Add a small delay

def run_app():
    !streamlit run app.py --server.port 8501 &

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(10)

# Generate a unique tunnel name
tunnel_name = f"streamlit-{random.randint(1000, 9999)}"

public_url = ngrok.connect(8501, name=tunnel_name)
print(f"🌐 Your Streamlit app is live here: {public_url.public_url}")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.243.195.67:8501

🌐 Your Streamlit app is live here: https://paltriest-jumblingly-bettye.ngrok-free.dev
